## __Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:__

* InvoiceNo — номер транзакции
* StockCode — код товара
* Description — описание товара
* Quantity — количество единиц товара, добавленных в заказ
* InvoiceDate — дата транзакции 
* UnitPrice — цена за единицу товара
* CustomerID — id клиента
* Country — страна, где проживает клиент

In [1]:
import pandas as pd

import requests
from urllib.parse import urlencode

#### __Загрузим данные__

In [2]:
def get_data(df_url, encoding, compression, sep=';'):
    """Функция для загрузки данных и их преобразования в датафрейм"""
    
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    
    # Получаем загрузочную ссылку
    final_url = base_url + urlencode(dict(public_key=df_url))
    response = requests.get(final_url)
    download_url = response.json()['href']

    # Загружаем файл и сохраняем его
    df = pd.read_csv(download_url, sep=',', encoding=encoding, compression=compression)
    
    return df

In [22]:
retail = get_data('https://disk.yandex.ru/d/q0n4CB1Rf0DxPg', 'ISO-8859-1', 'zip')
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


__Импортируйте пандас и прочитайте данные с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.__

In [5]:
retail_columns = retail.columns

__Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.__

In [8]:
retail.duplicated().sum() # всего 5268 дупликатов

5268

In [23]:
retail.drop_duplicates(inplace=True)

__Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled).__ 

__Сколько всего транзакций отменили пользователи?__

In [19]:
retail[retail['InvoiceNo'].str.startswith('C')].Description.count() # 9251 пользователь отменил заказ

9251

__Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.__

In [25]:
retail = retail[retail['Quantity'] > 0]
retail[retail['Quantity'] > 0].Description.count()

525462

__Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).__

In [61]:
germany_buyers = (
                    retail[retail['Country'] == 'Germany']
                    .groupby('CustomerID', as_index=False)
                    .agg({'InvoiceNo': pd.Series.nunique})
                    .rename(columns={'InvoiceNo': 'No_orders'})
                )
germany_buyers

,CustomerID,No_orders
0,12426.0,1
1,12427.0,3
2,12468.0,2
3,12471.0,30
4,12472.0,7
...,...,...
89,13814.0,2
90,13815.0,4
91,13816.0,2
92,13817.0,2


In [64]:
percent_quant_80 = germany_buyers.No_orders.quantile(0.8)

In [66]:
germany_top = germany_buyers.query("No_orders > @percent_quant_80").CustomerID.reset_index(drop=True)
germany_top

0     12471.0
1     12474.0
2     12476.0
3     12481.0
4     12500.0
5     12524.0
6     12569.0
7     12600.0
8     12619.0
9     12621.0
10    12626.0
11    12647.0
12    12662.0
13    12705.0
14    12708.0
15    12709.0
16    12712.0
17    12720.0
Name: CustomerID, dtype: float64

__Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.__

In [70]:
top_retail_germany = retail.query("CustomerID in @germany_top")
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany


__Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST__

In [75]:
(
    top_retail_germany[top_retail_germany['StockCode'] != 'POST']
    .groupby('StockCode')
    .agg({'InvoiceNo': 'count'})
    .idxmax()
) 

InvoiceNo    22326
dtype: object

__Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.__

In [78]:
retail['Revenue'] = retail['Quantity'] * retail['UnitPrice']
retail.Revenue

0         15.30
1         20.34
2         22.00
3         20.34
4         20.34
          ...  
541904    10.20
541905    12.60
541906    16.60
541907    16.60
541908    14.85
Name: Revenue, Length: 526054, dtype: float64

__Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)__

In [90]:
top_5_invoice = (
                    retail.groupby('InvoiceNo')['Revenue']
                    .sum()
                    .sort_values(ascending=False).head(5)
                )
top_5_invoice

InvoiceNo
581483    168469.60
541431     77183.60
574941     52940.94
576365     50653.91
556444     38970.00
Name: Revenue, dtype: float64

In [93]:
for index in top_5_invoice.index:
    print(index, end=', ')

581483, 541431, 574941, 576365, 556444, 